In [ ]:
exp_path = "/mnt/e/"

In [ ]:
systems = [
    "Experiment_nginx_energy",
    "Experiment_7z_energy",
    "Experiment_x264_energy",
    "Experiment_lrzip_energy",
    "Experiment_HSQLDB_energy",
    "Experiment_PostgreSQL_pervolution_energy",
    "Experiment_VP8_pervolution_energy",
    "Experiment_Apache_energy"
]

systems = [
    "Experiment_7z_energy",
    "Experiment_lrzip_energy",
    "Experiment_nginx_energy",
    "Experiment_PostgreSQL_pervolution_energy",
    "Experiment_VP8_pervolution_energy",
    "Experiment_x264_energy",
    "Experiment_HSQLDB_energy",
    "Experiment_Apache_energy"
]

systems = [
    "Experiment_7z_energy",
    "Experiment_lrzip_energy",
    "Experiment_VP8_pervolution_energy",
    "Experiment_PostgreSQL_pervolution_energy",
    "Experiment_nginx_energy",
    "Experiment_x264_energy",
    "Experiment_HSQLDB_energy",
    "Experiment_Apache_energy"
]

In [ ]:
def rename_strings(input_string):
    # Dictionary mit Umbenennungsregeln
    rename_dict = {
        "Experiment_x264_energy": "x264",
        "Experiment_VP8_pervolution_energy": "VP8",
        "Experiment_PostgreSQL_pervolution_energy": "Postgres",
        "Experiment_nginx_energy": "nginx",
        "Experiment_lrzip_energy": "lrzip",
        "Experiment_HSQLDB_energy": "HSQLDB",
        "Experiment_7z_energy": "7z",
        "Experiment_Apache_energy": "Apache",
        "Random_3": "Random 3",
        #"Random_2": "Random 2",
        "Random_2": "Random 2: 25",
        "Random_2_2": "Random 2: 50",
        "Random_2_3": "Random 2: 75",
        "Random_2_4": "Random 2: 150",
        "Random_1": "Random 1",
        "T-wise_2": "T-wise 2",
        "T-wise_3": "T-wise 3",
        "Distance_1": "Distance 1",
        "Distance_2": "Distance 2",
        "Distance_3": "Distance 3"
        # Weitere Umbenennungen hier hinzufügen
    }

    # Prüfen, ob der input_string im Dictionary existiert
    if input_string in rename_dict:
        return rename_dict[input_string]
    else:
        # Wenn keine Übereinstimmung gefunden wird, den originalen input_string zurückgeben
        return input_string

In [ ]:
method = "cqr_lgbm_quant_None"
#method = "jackknife_plus_lin_lasso_None"
#method = "baysian"

In [ ]:
#sampling_stategie = ["Random_1", "Random_2", "Random_3"]
sampling_stategie = ["Random_2", "Random_2_3", "Random_2_4"]
#sampling_stategie = ["OW", "T-wise_2", "T-wise_3"]
#sampling_stategie = ["Distance_1", "Distance_2", "Distance_3"]
alpha = 0.1

In [ ]:
def get_folder_names(path):
    folder_names = []
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        if os.path.isdir(item_path) and item not in ['$RECYCLE.BIN', 'System Volume Information']:
            folder_names.append(item)

    return folder_names

In [ ]:
import os
import json
import pandas as pd

col_system = []
col_Cov_err = []
col_coverage = []
col_sampling = []

for i in sampling_stategie:
    for s in systems:
        path = os.path.join(exp_path, s, i)
        runs = get_folder_names(path)

        for run in runs:
            for r in runs:
                json_name = f"{method}_result.txt"
                json_path = os.path.join(exp_path, s, i, r, json_name)
                try:
                    with open(json_path) as json_file:
                        data = json.load(json_file)
                        data["Cov_dif"] = abs(data['coverage'] - (1 - alpha))
                        col_Cov_err.append(data['Cov_dif'])
                        col_coverage.append(data["coverage"])
                        col_system.append(rename_strings(s))
                        col_sampling.append(rename_strings(i))
                except FileNotFoundError:
                    print(f"{s}, {i}: {r} has no Data for this ML-Model({method})")
                    col_Cov_err.append(None)
                    col_coverage.append(None)
                    col_system.append(rename_strings(s))
                    col_sampling.append(rename_strings(i))
daten = {
    "System": col_system,
    "Cov_err": col_Cov_err,
    "Coverage": col_coverage,
    "Sampling Strategie": col_sampling
}
result_df = pd.DataFrame(daten)

result_df


In [ ]:
sampling = "Random_25"
filtered_df = result_df.loc[result_df["Sampling Strategie"]==sampling]
filtered_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.boxplot(data=filtered_df, x='Cov_err', y='System', hue='System',legend=False, fill=True, palette='tab10')
#sns.kdeplot(data=result_df, x='Cov_err', hue='System', fill=True, common_norm=False, palette='tab10', cut=0)

plt.xlim(0, 0.15)

# Titel und Beschriftungen
plt.title(f"{rename_strings(sampling)}")
plt.xlabel('Coverage Error')
plt.ylabel('Density')

plt.show()

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
ax = sns.boxplot(data=filtered_df, x='Coverage', y='System', hue='System',legend=False, fill=True, palette='tab10')
plt.xlim(0.75, 1)
ax.axvline(x=(1 - 2 * alpha), color='r', linestyle='--')
ax.axvline(x=(1 - alpha), color='y', linestyle='--')

# Titel und Beschriftungen
plt.title(f"{rename_strings(sampling)}")
plt.xlabel('Coverage')
plt.ylabel('Density')


plt.show()

In [ ]:
g = sns.FacetGrid(result_df, col='Sampling Strategie', hue="System")
g.map(sns.kdeplot, 'Coverage', fill=True, cut=1)
plt.legend(title='Sampling', bbox_to_anchor=(1.1, 1), loc='upper left', borderaxespad=0., frameon=True)
g.set_titles(col_template="{col_name}")

In [ ]:
g = sns.FacetGrid(result_df, col='Sampling Strategie', height=3, aspect=1, )
#g.map(sns.boxplot,data=result_df, hue='System', x='Coverage', palette='tab10')
#g.map_dataframe(sns.boxplot, x="Coverage", y="System", hue='System',legend=False, fill=True, palette='tab10')
g.map_dataframe(sns.violinplot, x="Coverage", y="System", hue='System',legend=False, fill=True, palette='pastel')
axes = g.axes.flatten()
g.set_titles(col_template="{col_name}")
# Hinzufügen von vertikalen Linien zu jedem Plot



for ax in axes:
    ax.axvline(x=(1 - 2 * alpha), color='r', linestyle='--')
    ax.axvline(x=(1 - alpha), color='y', linestyle='--')

g.savefig(os.path.join(exp_path, "cqr-com-grid"), bbox_inches='tight')
